# Convolution 

## 一维卷积 / 1D Convolution

一维卷积常用于信号处理和时间序列分析。这里实现了一个简单的一维卷积函数。
1D convolution is commonly used in signal processing and time series analysis. Here we implement a simple 1D convolution function.

### 数学公式 / Mathematical Formula

对于1D卷积，数学表达式为：
For 1D convolution, the mathematical expression is:

$$(f * g)(t) = \sum_{i=-\infty}^{\infty} f(i) \cdot g(t-i)$$

其中：
Where:
- $f$ 是输入信号 / is the input signal
- $g$ 是卷积核 / is the convolution kernel  
- $t$ 是输出位置 / is the output position

### 关键概念 / Key Concepts

1. **填充 (Padding)**: 在信号两端添加零值，保持输出长度
   **Padding**: Adding zero values at signal ends to maintain output length
2. **步长 (Stride)**: 卷积核移动的步长（这里默认为1）
   **Stride**: Step size for kernel movement (default 1 here)
3. **感受野 (Receptive Field)**: 卷积核覆盖的输入区域
   **Receptive Field**: Input region covered by the kernel

### 需要澄清的问题 / Questions to Clarify

1. **边界处理**: 如何处理信号边界？使用零填充还是镜像填充？
   **Boundary Handling**: How to handle signal boundaries? Zero padding or mirror padding?

2. **步长选择**: 不同步长对特征提取有什么影响？
   **Stride Selection**: How does different stride affect feature extraction?

3. **卷积核设计**: 如何设计有效的卷积核？如何初始化？
   **Kernel Design**: How to design effective kernels? How to initialize them?

4. **计算复杂度**: 如何优化卷积计算？可以使用哪些加速技术？
   **Computational Complexity**: How to optimize convolution computation? What acceleration techniques can be used? 

In [26]:
def convolve(signal, kernel):
    """
    一维卷积函数 / 1D Convolution Function
    对输入信号应用卷积核进行卷积操作
    Apply convolution operation on input signal using kernel
    """
    output = []  # 输出结果列表 / Output result list
    kernel_size = len(kernel)  # 卷积核大小 / Kernel size
    padding = kernel_size // 2  # 计算零填充大小 / Calculate zero padding size
    padded_signal = [0] * padding + signal + [0] * padding  # 对信号进行零填充 / Apply zero padding to signal
    
    # 遍历信号的每个位置进行卷积 / Iterate through each position of signal for convolution
    for i in range(padding, len(signal) + padding):
        sum = 0  # 初始化累加器 / Initialize accumulator
        # 对卷积核的每个元素进行乘积累加 / Multiply and accumulate for each kernel element
        for j in range(kernel_size):
            sum += kernel[j] * padded_signal[i - padding + j]  # 卷积计算 / Convolution calculation
        output.append(sum)  # 将结果添加到输出列表 / Append result to output list
    
    return output  # 返回卷积结果 / Return convolution result


In [27]:
# 测试一维卷积函数 / Test 1D convolution function
signal = [1, 2, 3, 4, 5, 6]  # 输入信号 / Input signal
kernel = [1, 0, -1]          # 卷积核（边缘检测） / Kernel (edge detection)
output = convolve(signal, kernel)  # 执行卷积 / Perform convolution
print(output)  # 输出结果 / Output result


[-2, -2, -2, -2, -2, 5]


## 二维卷积 / 2D Convolution

二维卷积是卷积神经网络中的核心操作，用于处理图像数据。这里实现了一个完整的2D卷积函数。
2D convolution is the core operation in Convolutional Neural Networks, used for processing image data. Here we implement a complete 2D convolution function.

### 数学公式 / Mathematical Formula

对于2D卷积，数学表达式为：
For 2D convolution, the mathematical expression is:

$$(f * g)(x, y) = \sum_{i=-\infty}^{\infty} \sum_{j=-\infty}^{\infty} f(i, j) \cdot g(x-i, y-j)$$

其中：
Where:
- $f$ 是输入图像 / is the input image
- $g$ 是卷积核 / is the convolution kernel
- $(x, y)$ 是输出位置 / is the output position

### 关键概念 / Key Concepts

1. **多通道处理**: 同时处理多个输入通道和输出通道
   **Multi-channel Processing**: Simultaneously process multiple input and output channels
2. **空间卷积**: 在二维空间上进行卷积操作
   **Spatial Convolution**: Perform convolution operations in 2D space
3. **特征图**: 每个输出通道对应一个特征图
   **Feature Maps**: Each output channel corresponds to a feature map

### 需要澄清的问题 / Questions to Clarify

1. **边界处理**: 如何处理图像边界？使用零填充还是镜像填充？
   **Boundary Handling**: How to handle image boundaries? Zero padding or mirror padding?

2. **步长选择**: 不同步长对特征提取有什么影响？
   **Stride Selection**: How does different stride affect feature extraction?

3. **卷积核设计**: 如何设计有效的卷积核？如何初始化？
   **Kernel Design**: How to design effective kernels? How to initialize them?

4. **计算复杂度**: 如何优化卷积计算？可以使用哪些加速技术？
   **Computational Complexity**: How to optimize convolution computation? What acceleration techniques can be used? 

In [28]:
import numpy as np

def convolution_2d(image, kernel):
    """
    二维卷积函数 / 2D Convolution Function
    支持灰度图像和彩色图像的卷积操作
    Supports convolution operations for both grayscale and color images
    
    参数 / Parameters:
    - image: 输入图像，可以是2D(灰度)或3D(彩色) / Input image, can be 2D(grayscale) or 3D(color)
    - kernel: 卷积核，2D数组 / Convolution kernel, 2D array
    
    返回 / Returns:
    - output: 卷积结果，2D数组 / Convolution result, 2D array
    """
    # 获取输入图像和卷积核的尺寸 / Get the size of the input image and kernel
    (image_height, image_width) = image.shape[:2]  # 图像高度、宽度 / Image height, width
    (kernel_height, kernel_width) = kernel.shape[:2]  # 卷积核高度、宽度 / Kernel height, width
    
    # 计算'same'卷积所需的填充 / Calculate the padding needed for 'same' convolution
    pad_h = (kernel_height - 1) // 2  # 垂直方向填充 / Vertical padding
    pad_w = (kernel_width - 1) // 2   # 水平方向填充 / Horizontal padding
    
    # 根据图像类型进行不同的填充处理 / Different padding based on image type
    if len(image.shape) == 2:  # 灰度图像处理 / Grayscale image processing
        print("处理灰度图像 / Processing grayscale image")
        padded_image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), 'constant')
    else:  # 彩色图像处理 / Color image processing
        print("处理彩色图像 / Processing color image")
        padded_image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w), (0, 0)), 'constant')
    
    # 创建空的输出张量 / Create an empty output tensor
    output_height = image_height      # 输出高度 / Output height
    output_width = image_width        # 输出宽度 / Output width
    output = np.zeros((output_height, output_width))  # 初始化输出张量 / Initialize output tensor
    
    # 执行卷积操作 / Perform the convolution operation
    for i in range(output_height):           # 遍历输出高度 / Iterate through output height
        for j in range(output_width):        # 遍历输出宽度 / Iterate through output width
            # 根据图像类型进行不同的卷积计算 / Different convolution calculation based on image type
            if len(image.shape) == 2:  # 灰度图像卷积 / Grayscale image convolution
                # 直接进行2D卷积 / Direct 2D convolution
                output[i, j] = np.sum(kernel * padded_image[i:i+kernel_height, j:j+kernel_width])
            else:  # 彩色图像卷积 / Color image convolution
                # 对每个通道分别进行卷积，然后求和 / Convolve each channel separately, then sum
                image_patch = padded_image[i:i+kernel_height, j:j+kernel_width, :]  # 获取图像块 / Get image patch
                channel_sum = 0  # 初始化通道求和 / Initialize channel sum
                for c in range(image_patch.shape[2]):  # 遍历每个通道 / Iterate through each channel
                    channel_sum += np.sum(kernel * image_patch[:, :, c])  # 对每个通道进行卷积 / Convolve each channel
                output[i, j] = channel_sum  # 存储所有通道的卷积和 / Store sum of all channel convolutions
    
    return output  # 返回卷积结果 / Return convolution result


In [29]:
# ===== 灰度图像测试 / Grayscale Image Test =====
print("=" * 50)
print("灰度图像卷积测试 / Grayscale Image Convolution Test")
print("=" * 50)

# 创建2D灰度图像 / Create 2D grayscale image
grayscale_image = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])  # 3x3灰度图像 / 3x3 grayscale image
grayscale_kernel = np.array([[1, 0, -1], [0, 1, 0], [-1, 0, 1]])  # 3x3边缘检测核 / 3x3 edge detection kernel

# 执行灰度图像卷积 / Perform grayscale image convolution
grayscale_output = convolution_2d(grayscale_image, grayscale_kernel)
print('\n卷积结果 / Convolution Result:')
print(grayscale_output)

print(f"灰度图像形状 / Grayscale image shape: {grayscale_image.shape}")
print(f"卷积核形状 / Kernel shape: {grayscale_kernel.shape}")


print('\n灰度图像 / Grayscale Image:')
print(grayscale_image)

print('\n卷积核 / Kernel:')
print(grayscale_kernel)

print(f'\n输出形状 / Output shape: {grayscale_output.shape}')
print(f'输入形状 / Input shape: {grayscale_image.shape}')


灰度图像卷积测试 / Grayscale Image Convolution Test
处理灰度图像 / Processing grayscale image

卷积结果 / Convolution Result:
[[ 6.  4. -2.]
 [10.  5.  0.]
 [ 2.  6. 14.]]
灰度图像形状 / Grayscale image shape: (3, 3)
卷积核形状 / Kernel shape: (3, 3)

灰度图像 / Grayscale Image:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

卷积核 / Kernel:
[[ 1  0 -1]
 [ 0  1  0]
 [-1  0  1]]

输出形状 / Output shape: (3, 3)
输入形状 / Input shape: (3, 3)


In [30]:
# ===== 彩色图像测试 / Color Image Test =====
print("\n" + "=" * 50)
print("彩色图像卷积测试 / Color Image Convolution Test")
print("=" * 50)

# 创建3D彩色图像 / Create 3D color image
color_image = np.array([
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]],      # 第一行：RGB通道 / First row: RGB channels
    [[10, 11, 12], [13, 14, 15], [16, 17, 18]],  # 第二行：RGB通道 / Second row: RGB channels
    [[19, 20, 21], [22, 23, 24], [25, 26, 27]]   # 第三行：RGB通道 / Third row: RGB channels
])  # 3x3x3彩色图像 / 3x3x3 color image

color_kernel = np.array([[1, -1], [-1, 1]])  # 2x2卷积核 / 2x2 kernel

# 执行彩色图像卷积 / Perform color image convolution
color_output = convolution_2d(color_image, color_kernel)
print('\n卷积结果 / Convolution Result:')
print(color_output)

print(f"彩色图像形状 / Color image shape: {color_image.shape}")
print(f"卷积核形状 / Kernel shape: {color_kernel.shape}")

print('\n彩色图像 (第一个通道) / Color Image (First Channel):')
print(color_image[:, :, 0])

print('\n彩色图像 (第二个通道) / Color Image (Second Channel):')
print(color_image[:, :, 1])

print('\n彩色图像 (第三个通道) / Color Image (Third Channel):')
print(color_image[:, :, 2])

print('\n卷积核 / Kernel:')
print(color_kernel)

print(f'\n输出形状 / Output shape: {color_output.shape}')
print(f'输入形状 / Input shape: {color_image.shape[:2]} (空间维度) / (spatial dimensions)')



彩色图像卷积测试 / Color Image Convolution Test
处理彩色图像 / Processing color image

卷积结果 / Convolution Result:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
彩色图像形状 / Color image shape: (3, 3, 3)
卷积核形状 / Kernel shape: (2, 2)

彩色图像 (第一个通道) / Color Image (First Channel):
[[ 1  4  7]
 [10 13 16]
 [19 22 25]]

彩色图像 (第二个通道) / Color Image (Second Channel):
[[ 2  5  8]
 [11 14 17]
 [20 23 26]]

彩色图像 (第三个通道) / Color Image (Third Channel):
[[ 3  6  9]
 [12 15 18]
 [21 24 27]]

卷积核 / Kernel:
[[ 1 -1]
 [-1  1]]

输出形状 / Output shape: (3, 3)
输入形状 / Input shape: (3, 3) (空间维度) / (spatial dimensions)


In [31]:
# ===== 修复后的彩色图像测试 / Fixed Color Image Test =====
print("\n" + "=" * 50)
print("修复后的彩色图像卷积测试 / Fixed Color Image Convolution Test")
print("=" * 50)

# 创建简化的3D彩色图像 / Create simplified 3D color image
simple_color_image = np.array([
    [[1, 2], [3, 4]],  # 第一个通道 / First channel
    [[5, 6], [7, 8]]   # 第二个通道 / Second channel
])  # 2x2x2彩色图像 / 2x2x2 color image

simple_kernel = np.array([[1, 0], [0, -1]])  # 2x2卷积核 / 2x2 kernel

# 执行彩色图像卷积 / Perform color image convolution
simple_color_output = convolution_2d(simple_color_image, simple_kernel)

print('\n卷积结果 / Convolution Result:')
print(simple_color_output)

print(f"彩色图像形状 / Color image shape: {simple_color_image.shape}")
print(f"卷积核形状 / Kernel shape: {simple_kernel.shape}")

print('\n彩色图像 (第一个通道) / Color Image (First Channel):')
print(simple_color_image[:, :, 0])

print('\n彩色图像 (第二个通道) / Color Image (Second Channel):')
print(simple_color_image[:, :, 1])

print('\n卷积核 / Kernel:')
print(simple_kernel)


print(f'\n输出形状 / Output shape: {simple_color_output.shape}')
print(f'输入形状 / Input shape: {simple_color_image.shape[:2]} (空间维度) / (spatial dimensions)')



修复后的彩色图像卷积测试 / Fixed Color Image Convolution Test
处理彩色图像 / Processing color image

卷积结果 / Convolution Result:
[[-12.  -8.]
 [ -4.   0.]]
彩色图像形状 / Color image shape: (2, 2, 2)
卷积核形状 / Kernel shape: (2, 2)

彩色图像 (第一个通道) / Color Image (First Channel):
[[1 3]
 [5 7]]

彩色图像 (第二个通道) / Color Image (Second Channel):
[[2 4]
 [6 8]]

卷积核 / Kernel:
[[ 1  0]
 [ 0 -1]]

输出形状 / Output shape: (2, 2)
输入形状 / Input shape: (2, 2) (空间维度) / (spatial dimensions)


## 彩色图像卷积原理 / Color Image Convolution Principle

### 问题分析 / Problem Analysis

原始代码中的错误：
```python
# 错误的做法 / Wrong approach
output[i, j] = np.sum(kernel * padded_image[i:i+kernel_height, j:j+kernel_width, :])
```

**问题**: 试图将2D卷积核 `(2,2)` 与3D图像块 `(2,2,3)` 直接相乘，导致广播错误。

### 解决方案 / Solution

修复后的代码：
```python
# 正确的做法 / Correct approach
image_patch = padded_image[i:i+kernel_height, j:j+kernel_width, :]  # 获取3D图像块
channel_sum = 0
for c in range(image_patch.shape[2]):  # 遍历每个通道
    channel_sum += np.sum(kernel * image_patch[:, :, c])  # 对每个通道分别卷积
output[i, j] = channel_sum  # 存储所有通道的卷积和
```

### 工作原理 / How It Works

1. **获取图像块**: 从3D图像中提取 `(kernel_height, kernel_width, channels)` 的图像块
2. **通道分离**: 将3D图像块分解为多个2D通道
3. **逐通道卷积**: 对每个2D通道分别应用2D卷积核
4. **结果求和**: 将所有通道的卷积结果相加

### 数学表示 / Mathematical Representation

对于彩色图像卷积：
$$(f * g)(x, y) = \sum_{c=0}^{C-1} \sum_{i} \sum_{j} f_c(i, j) \cdot g(x-i, y-j)$$

其中：
- $f_c$ 是第c个通道的图像
- $g$ 是2D卷积核
- $C$ 是通道数


In [32]:
# ===== 灰度vs彩色图像对比 / Grayscale vs Color Image Comparison =====
print("\n" + "=" * 60)
print("灰度图像 vs 彩色图像对比 / Grayscale vs Color Image Comparison")
print("=" * 60)

# 创建相同的空间结构但不同通道数的图像 / Create images with same spatial structure but different channels
print("1. 灰度图像 (2D) / Grayscale Image (2D):")
gray_img = np.array([[1, 2], [3, 4]])  # 2x2灰度图像 / 2x2 grayscale image
print(f"   形状 / Shape: {gray_img.shape}")
print(f"   数据 / Data:\n{gray_img}")

print("\n2. 彩色图像 (3D) / Color Image (3D):")
color_img = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])  # 2x2x2彩色图像 / 2x2x2 color image
print(f"   形状 / Shape: {color_img.shape}")
print(f"   第一个通道 / First channel:\n{color_img[:, :, 0]}")
print(f"   第二个通道 / Second channel:\n{color_img[:, :, 1]}")

# 使用相同的卷积核 / Use the same kernel
test_kernel = np.array([[1, 0], [0, -1]])

print(f"\n3. 卷积核 / Kernel:")
print(f"   形状 / Shape: {test_kernel.shape}")
print(f"   数据 / Data:\n{test_kernel}")

# 分别对两种图像进行卷积 / Perform convolution on both image types
print(f"\n4. 卷积结果对比 / Convolution Results Comparison:")

# 灰度图像卷积 / Grayscale convolution
gray_result = convolution_2d(gray_img, test_kernel)
print(f"   灰度图像卷积结果 / Grayscale result:\n{gray_result}")

# 彩色图像卷积 / Color convolution  
color_result = convolution_2d(color_img, test_kernel)
print(f"   彩色图像卷积结果 / Color result:\n{color_result}")

print(f"\n5. 关键区别 / Key Differences:")
print(f"   - 灰度图像: 输入2D，输出2D / Grayscale: 2D input → 2D output")
print(f"   - 彩色图像: 输入3D，输出2D / Color: 3D input → 2D output")
print(f"   - 彩色图像卷积时对所有通道求和 / Color convolution sums over all channels")



灰度图像 vs 彩色图像对比 / Grayscale vs Color Image Comparison
1. 灰度图像 (2D) / Grayscale Image (2D):
   形状 / Shape: (2, 2)
   数据 / Data:
[[1 2]
 [3 4]]

2. 彩色图像 (3D) / Color Image (3D):
   形状 / Shape: (2, 2, 2)
   第一个通道 / First channel:
[[1 3]
 [5 7]]
   第二个通道 / Second channel:
[[2 4]
 [6 8]]

3. 卷积核 / Kernel:
   形状 / Shape: (2, 2)
   数据 / Data:
[[ 1  0]
 [ 0 -1]]

4. 卷积结果对比 / Convolution Results Comparison:
处理灰度图像 / Processing grayscale image
   灰度图像卷积结果 / Grayscale result:
[[-3. -2.]
 [-1.  0.]]
处理彩色图像 / Processing color image
   彩色图像卷积结果 / Color result:
[[-12.  -8.]
 [ -4.   0.]]

5. 关键区别 / Key Differences:
   - 灰度图像: 输入2D，输出2D / Grayscale: 2D input → 2D output
   - 彩色图像: 输入3D，输出2D / Color: 3D input → 2D output
   - 彩色图像卷积时对所有通道求和 / Color convolution sums over all channels


## 总结 / Summary

### 图像类型识别 / Image Type Identification

我们的2D卷积函数能够自动识别和处理两种类型的图像：

**1. 灰度图像 (Grayscale Images)**
- **维度**: 2D (高度 × 宽度)
- **检测方法**: `len(image.shape) == 2`
- **处理方式**: 直接进行2D卷积
- **示例**: `image.shape = (3, 3)` → 3×3灰度图像

**2. 彩色图像 (Color Images)**  
- **维度**: 3D (高度 × 宽度 × 通道数)
- **检测方法**: `len(image.shape) == 3`
- **处理方式**: 对所有通道进行卷积后求和
- **示例**: `image.shape = (3, 3, 3)` → 3×3×3彩色图像

### 关键区别 / Key Differences

| 特征 / Feature | 灰度图像 / Grayscale | 彩色图像 / Color |
|---|---|---|
| **输入维度** / Input Dimensions | 2D | 3D |
| **输出维度** / Output Dimensions | 2D | 2D |
| **通道处理** / Channel Processing | 单通道 / Single channel | 多通道求和 / Multi-channel sum |
| **填充方式** / Padding | 2D填充 / 2D padding | 3D填充 / 3D padding |
| **卷积计算** / Convolution | 直接2D卷积 / Direct 2D | 通道求和 / Channel sum |

### 应用场景 / Use Cases

- **灰度图像**: 医学影像、卫星图像、文档扫描
- **彩色图像**: 自然图像、视频帧、RGB数据
